In [ ]:
import numpy as np
import random
from numpy.typing import NDArray
from scipy.stats import qmc

class Algoritmo_Genetico:
    def __init__(self, tamanho_populacao: int, escolhas_discretas: list[list] = [], range_var_continuas: list[list[float]] = [], seed: int | None = None):
        np.random.seed(seed)
        random.seed(seed)

        self.escolhas_discretas = np.array(escolhas_discretas)
        self.limites            = np.array(range_var_continuas)

        self.qtd_var_discretas = len(self.escolhas_discretas)
        self.qtd_var_continuas = len(self.limites)
        self.tem_var_continuas = self.qtd_var_continuas > 0
        self.tem_var_discretas = self.qtd_var_discretas > 0

        if not self.tem_var_continuas and not self.tem_var_discretas:
            raise ValueError("Nenhuma variável foi definida.")
        if tamanho_populacao < 2:
            raise ValueError("A população deve ter pelo menos 2 indivíduos.")
        if self.qtd_var_discretas > 0:
            if np.any([len(opcoes) for opcoes in self.escolhas_discretas] == 1):
                raise ValueError("Uma das variáveis discretas possui somente uma opção.")
        if self.qtd_var_continuas > 0:
            if np.any([limite[1] - limite[0] for limite in self.limites] == 0):
                raise ValueError("Uma das variáveis contínuas possui limite inferior igual ao limite superior.")

        self.qtd_total_var     = self.qtd_var_discretas + self.qtd_var_continuas
        self.tamanho_populacao = tamanho_populacao

        self.genes_populacao   = self.inicializar_genes(tamanho_populacao)        
        self.populacao         = self.avaliar_genes(self.genes_populacao)
        self.melhores          = [max(self.populacao, key=lambda x: x['fitness'])]

    def funcao_fitness(self, genes_avaliar: dict[str, NDArray]) -> float:
        """
        Essa é a função objetivo, precisa ser definida para o funcionamento do AG.
        Lembrar que a função deve ser maximizada.
        """
        genes_discretos = genes_avaliar['genes_discretos']
        genes_continuos = genes_avaliar['genes_continuos']

        return 0

    def genes_iniciais_continuos(self, quantidade_individuos: int) -> NDArray:
        """
        Usa a distribuição de Latin Hypercube para garantir boa variabilidade genética inicial para as variáveis contínuas.
        Retorna uma NDArray de listas vazias se não forem definidas variáveis contínuas.
        """
        if not self.tem_var_continuas:
            return np.empty((quantidade_individuos, 0))

        else:
            sampler = qmc.LatinHypercube(d=self.qtd_var_continuas)
            sample_gerado = sampler.random(n=quantidade_individuos)

            limites_inferiores = np.min(self.limites, axis=1)
            limites_superiores = np.max(self.limites, axis=1)

            genes = qmc.scale(sample_gerado, l_bounds=limites_inferiores, u_bounds=limites_superiores)

        return np.array(genes)
    
    def genes_iniciais_discretos(self, quantidade_individuos: int) -> NDArray:
        """
        Usa escolhas aleatórias para garantir boa variabilidade genética inicial para as variáveis discretas.
        Retorna uma NDArray de listas vazias se não forem definidas variáveis discretas.
        """
        if not self.tem_var_discretas:
            return np.empty((quantidade_individuos, 0))
    
        genes = []

        for _ in range(quantidade_individuos):
            individuo = [np.random.choice(opcoes) for opcoes in self.escolhas_discretas]
            genes.append(individuo)

        return np.array(genes)
    
    def inicializar_genes(self, tamanho_populacao: int) -> list[dict[str, NDArray]]:
        """
        inicializa os genes contínuos e discretos da população, criando indivíduos sem nota.
        Já considera se não forem definidas variáveis contínuas ou discretas.
        """
        genes_discretos = self.genes_iniciais_discretos(tamanho_populacao)
        genes_continuos = self.genes_iniciais_continuos(tamanho_populacao)
        genes_populacao = []

        for i in range(tamanho_populacao):
            genes_populacao.append({'genes_discretos': genes_discretos[i], 'genes_continuos': genes_continuos[i]})
        
        return genes_populacao

    def avaliar_genes(self, genes_avaliar: list[dict[str, NDArray]]) -> list[dict]:
        """Adiciona a entrada 'fitness' em cada gene, com o seu valor avaliado pela self.funcao_fitness."""
        for genes in genes_avaliar:
            fitness = self.funcao_fitness(genes)
            genes['fitness'] = fitness
        
        return genes_avaliar

    def crossover_discreto(self, genes_discretos_pai: NDArray, genes_discretos_mae: NDArray) -> NDArray:
        """
        Usa escolhas aleatórias entre os genes dos pais para gerar os genes do cruzamento.
        """
        genes_filho = []

        for i in range(len(self.escolhas_discretas)):
            gene_escolhido = np.random.choice([genes_discretos_pai[i], genes_discretos_mae[i]])
            genes_filho.append(gene_escolhido)

        return np.array(genes_filho)

    def crossover_continuo(self, genes_continuos_pai: NDArray, genes_continuos_mae: NDArray, eta_c: float) -> NDArray:
        """
        Usa SBX (Simulated Binary Crossover) para calcular os genes contínuos do cruzamento dos pais.
        O SBX também garante a possibilidade de extrapolação dos para valores além dos genes dos pais.
        """
        genes_filho = []
        u = np.random.uniform(0, 1, self.qtd_var_continuas)

        # calcula beta
        beta_q = np.where(u <= 0.5,
            (2*u) ** (1 / (eta_c + 1)),
            (1 / (2 * (1-u))) ** (1 / (eta_c + 1))
        )

        # calcula o valor do gene contínuo usando SBX
        for i in range(self.qtd_var_continuas):
            beta = beta_q[i]
            gene1 = (0.5 * ((1+beta) * genes_continuos_pai[i] + (1-beta) * genes_continuos_mae[i]))
            gene2 = (0.5 * ((1-beta) * genes_continuos_pai[i] + (1+beta) * genes_continuos_mae[i]))

            gene_escolhido = np.random.choice([gene1, gene2])
            gene_escolhido = np.clip(gene_escolhido, np.min(self.limites[i]), np.max(self.limites[i]))

            genes_filho.append(gene_escolhido)
        
        return np.array(genes_filho)

    def crossover(self, pai: dict, mae: dict, eta_c: float) -> dict[str, NDArray]:
        """
        Realiza cruzamento entre dois indivíduos e retorna os genes do cruzamento.
        Já considera se não houverem variáveis contínuas ou discretas.
        """
        if self.tem_var_discretas:
            genes_discretos_pai   = pai['genes_discretos'].copy()
            genes_discretos_mae   = mae['genes_discretos'].copy()
            genes_discretos_filho = self.crossover_discreto(genes_discretos_pai, genes_discretos_mae)
        else:
            genes_discretos_filho = np.empty(0)
        
        if self.tem_var_continuas:
            genes_continuos_pai   = pai['genes_continuos'].copy()
            genes_continuos_mae   = mae['genes_continuos'].copy()
            genes_continuos_filho = self.crossover_continuo(genes_continuos_pai, genes_continuos_mae, eta_c)
        else:
            genes_continuos_filho = np.empty(0)

        genes_filho = {
            'genes_discretos': genes_discretos_filho,
            'genes_continuos': genes_continuos_filho
            }

        return genes_filho

    def mutacao_discreta(self, genes_originais: NDArray) -> NDArray:
        """
        Altera um dos genes_originais para uma escolha aleatória diferente, usando a lista self.escolhas_discretas.
        Usar somente se tiverem variáveis discretas para escolha.
        """
        gene_mutar = np.random.randint(self.qtd_var_discretas)
        genes_mutados = genes_originais.copy()

        while genes_mutados[gene_mutar] == genes_originais[gene_mutar]:
            genes_mutados[gene_mutar] = np.random.choice(self.escolhas_discretas[gene_mutar])
        
        return genes_mutados
    
    def mutacao_continua(self, genes_originais: NDArray, proporcao_desvio: float) -> NDArray:
        """
        Altera um dos genes_originais para um valor aleatório normalmente distribuído em volta de seu valor original.
        Usar somente se tiverem variáveis contínuas para escolha.
        """
        gene_mutar = np.random.randint(self.qtd_var_continuas)
        genes_mutados = genes_originais.copy()
        desvio_padrao = proporcao_desvio * (np.max(self.limites[gene_mutar]) - np.min(self.limites[gene_mutar]))

        genes_mutados[gene_mutar] = np.random.normal(genes_originais[gene_mutar], desvio_padrao)
        genes_mutados[gene_mutar] = np.clip(genes_mutados[gene_mutar], np.min(self.limites[gene_mutar]), np.max(self.limites[gene_mutar]))

        return genes_mutados

    def mutacao(self, individuo: dict, proporcao_desvio: float = 0.1) -> dict[str, NDArray]:
        """
        Realiza mutação de um dos genes_originais.
        já considera se não houverem variáveis contínuas ou discretas.
        """
        genes_discretos = individuo['genes_discretos']
        genes_continuos = individuo['genes_continuos']

        genes_discretos_mutados = genes_discretos
        genes_continuos_mutados = genes_continuos

        if self.tem_var_discretas and not self.tem_var_continuas:
            genes_discretos_mutados = self.mutacao_discreta(genes_discretos)
            
        elif not self.tem_var_discretas and self.tem_var_continuas:
            genes_continuos_mutados = self.mutacao_continua(genes_continuos, proporcao_desvio)

        else:
            qual_gene_mutar = np.random.randint(self.qtd_var_discretas + self.qtd_var_continuas)

            if qual_gene_mutar < self.qtd_var_discretas:
                genes_discretos_mutados = self.mutacao_discreta(genes_discretos)

            else:
                genes_continuos_mutados = self.mutacao_continua(genes_continuos, proporcao_desvio)

        genes_mutados = {
            'genes_discretos': genes_discretos_mutados,
            'genes_continuos': genes_continuos_mutados
            }

        return genes_mutados

    def roleta(self, tamanho_roleta: int) -> dict:
        """Escolhe aleatoriamente tamanho_roleta candidatos de self.populacao e usa seus valores de fitness para escolher um"""
        candidatos = random.sample(self.populacao, tamanho_roleta)

        notas = np.array([candidato['fitness'] for candidato in candidatos])
        notas = notas - np.min(notas) + 1e-9

        if np.all(notas == 0):
            probabilidade_escolha = np.ones_like(notas) / len(notas)

        else:
            probabilidade_escolha = notas / np.sum(notas)

        vencedor = random.choices(candidatos, weights=probabilidade_escolha, k=1)[0]
        return vencedor

    def treinar(self, qtd_geracoes: int, taxa_crossover: float, taxa_mutacao: float, tamanho_roleta: int, paciencia: int, proporcao_desvio: float = 0.1, eta_c_inicial: float = 2, eta_c_max: float = 20) -> list[dict[str, NDArray]]:
        geracoes_sem_melhora = 0
        
        eta_c = eta_c_inicial
        passo_eta_c = (eta_c_max - eta_c_inicial) / paciencia

        for geracao in range(qtd_geracoes):
            # Elitismo
            melhor = self.melhores[-1]
            self.genes_populacao[-1] = {
                'genes_discretos': melhor['genes_discretos'].copy(),
                'genes_continuos': melhor['genes_continuos'].copy()
            }
            
            # Cria uma população nova a partir da população da geração anterior
            for i in range(self.tamanho_populacao - 1):
                pai = self.roleta(tamanho_roleta)
                mae = self.roleta(tamanho_roleta)

                # Crossover ou clonagem
                if np.random.random() < taxa_crossover:
                    genes_filho = self.crossover(pai, mae, eta_c)
                else:
                    genes_filho = {
                        'genes_discretos': random.choice([pai, mae])['genes_discretos'].copy(),
                        'genes_continuos': random.choice([pai, mae])['genes_continuos'].copy()
                    }

                # Mutação
                if np.random.random() < taxa_mutacao:
                    genes_filho = self.mutacao(genes_filho, proporcao_desvio)
                
                self.genes_populacao[i] = genes_filho

            # Avalia a nova população
            self.populacao = self.avaliar_genes(self.genes_populacao)
            self.melhores.append(max(self.populacao, key=lambda x: x['fitness']))

            # Se melhorou, reseta a contagem
            if self.melhores[-1]['fitness'] > self.melhores[-2]['fitness']:
                geracoes_sem_melhora = 0

            # Senão, soma mais um na contagem e aplica um equivalente do reduce_lr_on_plateau para variáveis contínuas
            else:
                eta_c = min(eta_c_max, eta_c + passo_eta_c)
                geracoes_sem_melhora += 1

            # Early stopping
            if geracoes_sem_melhora >= paciencia:
                break

        print(f'Treino realizado com sucesso em {geracao + 1} gerações!')
        return self.melhores